In [12]:
'''poker engine'''

# pylint: disable=E1101, E1601, W0612

import gc
import mysql.connector
from pypokerengine.api.emulator import Emulator

import funcs_db

In [13]:
gc.collect()

878

# OPEN CONNECTION TO POKER DB 

In [14]:
poker_db = mysql.connector.connect(user='root', host='127.0.0.1', database='poker_version2')
sql_path = 'c:\\Users\\adam.sohonyai\\Documents\\GitHub\\poker_model\\sql\\version2\\'

## Delete data from database tables 

In [15]:
tables = funcs_db.sql_select_tables(poker_db=poker_db)
for table in tables:
    funcs_db.sql_delete_all(poker_db=poker_db, table=table)

# Initialization

In [16]:
emulator = Emulator()

## Create game and table

In [17]:
player_num = 6
max_round = 1
small_blind_amount = 10
ante_amount = 0

stack = 2000
players_info = {
    'uuid-0': {'name': 'player0', 'stack': stack},
    'uuid-1': {'name': 'player1', 'stack': stack},
    'uuid-2': {'name': 'player2', 'stack': stack},
    'uuid-3': {'name': 'player3', 'stack': stack},
    'uuid-4': {'name': 'player4', 'stack': stack},
    'uuid-5': {'name': 'player5', 'stack': stack}
}

ID = funcs_db.sql_games_max_id(poker_db=poker_db) + 1

In [18]:
emulator.set_game_rule(player_num=player_num, max_round=max_round, small_blind_amount=small_blind_amount, \
                       ante_amount=ante_amount)
initial_state = emulator.generate_initial_game_state(players_info)
for i, player in enumerate(initial_state['table'].seats.players):

    funcs_db.sql_insert_games(poker_db=poker_db, index=ID, player_num=player_num, small_blind_amount=small_blind_amount, \
                              ante_amount=ante_amount, uuid=player.uuid, name=player.name, stack=player.stack, position=i, \
                              card1='', card2='', hand_db_format='')

In [19]:
game_state, events = emulator.start_new_round(initial_state)
for i, player in enumerate(game_state['table'].seats.players):
    
    card_list = [player.hole_card[0].RANK_MAP[player.hole_card[0].rank] + \
                 player.hole_card[0].SUIT_MAP[player.hole_card[0].suit], \
                 player.hole_card[1].RANK_MAP[player.hole_card[1].rank] + \
                 player.hole_card[1].SUIT_MAP[player.hole_card[1].suit]
    ]
    card_list.sort()
    card1 = card_list[0]
    card2 = card_list[1]
    if card1[-1] == card2[-1]:
        hand_db_format = card1[:-1] + card2[:-1] + 's'
    else:
        hand_db_format = card1[:-1] + card2[:-1] + 'o'
    
    funcs_db.sql_update_games_cards(poker_db=poker_db, index=ID, uuid=player.uuid, \
                                    card1=card1, card2=card2, hand_db_format=hand_db_format)

## Small blind and big blind moves

In [20]:
nr = 0
pot = 0
for i, player in enumerate(game_state['table'].seats.players):
    
    try:
        if player.serialize()[4][0]['action'] in ['SMALLBLIND', 'BIGBLIND']:
            if player.serialize()[4][0]['action'] == 'SMALLBLIND':
                amount = small_blind_amount
            else:
                amount = 2 * small_blind_amount
            funcs_db.sql_insert_history(poker_db=poker_db, phase='preflop', nr=nr, uuid=player.serialize()[1], position=i, \
                                        stack=2000, pot=pot, flop1='', flop2='', flop3='', turn='', river='', \
                                        action=player.serialize()[4][0]['action'], amount=amount, \
                                        new_stack=2000 - amount, new_pot=pot + amount)

            nr = nr + 1
            pot = pot + amount
    except:
        pass

# Preflop phase

In [21]:
log = list()
while events[-1]['type'] != 'event_game_finish':
    
    log.append(events[-1])
    
    phase = events[-1]['round_state']['street']
    position = events[-1]['round_state']['next_player']
    uuid = events[-1]['uuid']
    for item in events[-1]['round_state']['seats']:

        if item['uuid'] == events[-1]['uuid']:
            stack = item['stack']

    pot = events[-1]['round_state']['pot']['main']['amount']
    valid_actions = events[-1]['valid_actions']

    action, bet_amount = funcs_db.decision_point_based_action(poker_db=poker_db, phase=phase, nr=nr, position=position, \
                                                              stack=stack, pot=pot, valid_actions=valid_actions)
    funcs_db.sql_insert_history(poker_db=poker_db, phase=phase, nr=nr, uuid=uuid, position=position, stack=stack, pot=pot, \
                                flop1='', flop2='', flop3='', turn='', river='', \
                                action=action.upper(), amount=bet_amount, new_stack=stack - bet_amount, new_pot=pot + bet_amount)
    game_state, events = emulator.apply_action(game_state=game_state, action=action, bet_amount=bet_amount)

    nr = nr + 1

In [23]:
log

[{'type': 'event_ask_player',
  'uuid': 'uuid-3',
  'valid_actions': [{'action': 'fold', 'amount': 0},
   {'action': 'call', 'amount': 20},
   {'action': 'raise', 'amount': {'min': 30, 'max': 2000}}],
  'round_state': {'street': 'preflop',
   'pot': {'main': {'amount': 30}, 'side': []},
   'community_card': [],
   'dealer_btn': 0,
   'next_player': 3,
   'small_blind_pos': 1,
   'big_blind_pos': 2,
   'round_count': 1,
   'small_blind_amount': 10,
   'seats': [{'name': 'player0',
     'uuid': 'uuid-0',
     'stack': 2000,
     'state': 'participating'},
    {'name': 'player1',
     'uuid': 'uuid-1',
     'stack': 1990,
     'state': 'participating'},
    {'name': 'player2',
     'uuid': 'uuid-2',
     'stack': 1980,
     'state': 'participating'},
    {'name': 'player3',
     'uuid': 'uuid-3',
     'stack': 2000,
     'state': 'participating'},
    {'name': 'player4',
     'uuid': 'uuid-4',
     'stack': 2000,
     'state': 'participating'},
    {'name': 'player5',
     'uuid': 'uuid-5